In [85]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# LCMS Model Application

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/redcastle-resources/lcms-training/blob/main/5-Model_Application.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/redcastle-resources/lcms-training/blob/main/5-Model_Application.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/redcastle-resources/lcms-training/blob/main/5-Model_Application.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview


This notebook builds and applies the models for creating LCMS outputs


### Objective

In this tutorial, you learn how to apply LCMS models over large areas

This tutorial uses the following Google Cloud services:

- `Google Earth Engine`

The steps performed include:

- Training LCMS models and looking at outputs
- Learning about different model modes in GEE
- Applying LCMS models and exporting outputs


In [7]:
#Module imports
#!python -m pip install geeViz --upgrade
try:
    import geeViz.getImagesLib as getImagesLib
except:
    !python -m pip install geeViz
    import geeViz.getImagesLib as getImagesLib

import geeViz.changeDetectionLib as changeDetectionLib
import geeViz.assetManagerLib as aml
import geeViz.taskManagerLib as tml
import geeViz.gee2Pandas as g2p
import inspect,operator
import matplotlib.pyplot as plt
import pandas as pd  
# from IPython.display import IFrame,display, HTML
ee = getImagesLib.ee
Map = getImagesLib.Map

# Can set the port used for viewing map outputs
Map.port = 1235
print('Done')


Done


## Before you begin

### Set your current URL under `workbench_url`
* This will be in your URL/search bar at the top of the browser window you are currently in
* It will look something like `https://1234567890122-dot-us-west3.notebooks.googleusercontent.com/`

### Set a folder to use for all exports under `export_path_root` 
* It will be something like `projects/projectID/assets/someFolder`
* This folder does not have to already exist. If it does not exist, it will be created

In [2]:
workbench_url = 'https://23dcc4ff89e513fb-dot-us-west3.notebooks.googleusercontent.com/'
export_path_root  = 'projects/rcr-gee/assets/lcms-training'

print('Done')

Done


In [41]:
# Bring in all folders/collections that are needed
# These must already exist as they are created in previous notebooks
export_composite_collection = f'{export_path_root}/lcms-training_module-2_composites'
export_landTrendr_collection = f'{export_path_root}/lcms-training_module-3_landTrendr'
export_ccdc_collection = f'{export_path_root}/lcms-training_module-3_CCDC'
export_timeSync_folder = f'{export_path_root}/lcms-training_module-4_timeSync'

export_rawLCMSOutputs_collection = f'{export_path_root}/lcms-training_module-5_rawLCMSOutputs'



aml.create_asset(export_rawLCMSOutputs_collection, asset_type = ee.data.ASSET_TYPE_IMAGE_COLL)

# Currently geeView within Colab uses a different project to authenticate through, so you may need to make your asset public to view from within Colab
aml.updateACL(export_rawLCMSOutputs_collection,writers = [],all_users_can_read = True,readers = [])

print('Done')

Found the following sub directories:  ['lcms-training', 'lcms-training_module-5_rawLCMSOutputs']
Will attempt to create them if they do not exist
Asset projects/rcr-gee/assets/lcms-training already exists
Asset projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs already exists
Updating permissions for:  projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs
Done


In [5]:
# First we have to set up many of the same pieces as Module 4

Map.proxy_url = workbench_url
# First, let's bring in the exported training data and set up some parameters
# Set the projection
crs = getImagesLib.common_projections['NLCD_CONUS']['crs']
transform  = getImagesLib.common_projections['NLCD_CONUS']['transform']
scale = None

# Bring in annualized extracted TimeSync data

# Now lets bring in all training data and prep it for modeling
assets = ee.data.listAssets({'parent': export_timeSync_folder})['assets']

# You may need to change the permissions for viewing model outputs in geeViz
# Uncomment this if needed
# for asset in assets:aml.updateACL(asset['name'],writers = [],all_users_can_read = True,readers = [])

# Read in each year of extracted TimsSync data
training_data = ee.FeatureCollection([ee.FeatureCollection(asset['name']) for asset in assets]).flatten()

# Bring in existing LCMS data for the class names, numbers, and colors
lcms_viz_dict = ee.ImageCollection("USFS/GTAC/LCMS/v2020-6").first().toDictionary().getInfo()
                                             
print('LCMS class code, names, and colors:',lcms_viz_dict)


# Filter out any non null values (any training plot with missing predictor data will cause the model to fail entirely)
# training_data = training_data.filter(ee.Filter.notNull(predictor_field_names))
print('Training data first plot:',training_data.first().getInfo())


training_years = [int(yr) for yr in training_data.aggregate_histogram('YEAR').keys().getInfo()]
plot_ids = training_data.aggregate_histogram('PLOTID').keys().getInfo()
print('Training years:',training_years)
print('Unique Plot ID Count:',len(plot_ids))
print('Total Training Plots:', training_data.size().getInfo())
print('Training Data Attributes:',training_data.first().propertyNames().getInfo())

print('Done')

LCMS class code, names, and colors: {'Change_class_names': ['Stable', 'Slow Loss', 'Fast Loss', 'Gain', 'Non-Processing Area Mask'], 'Change_class_palette': ['3d4551', 'f39268', 'd54309', '00a398', '1b1716'], 'Change_class_values': [1, 2, 3, 4, 5], 'Land_Cover_class_names': ['Trees', 'Tall Shrubs & Trees Mix (SEAK Only)', 'Shrubs & Trees Mix', 'Grass/Forb/Herb & Trees Mix', 'Barren & Trees Mix', 'Tall Shrubs (SEAK Only)', 'Shrubs', 'Grass/Forb/Herb & Shrubs Mix', 'Barren & Shrubs Mix', 'Grass/Forb/Herb', 'Barren & Grass/Forb/Herb Mix', 'Barren or Impervious', 'Snow or Ice', 'Water', 'Non-Processing Area Mask'], 'Land_Cover_class_palette': ['005e00', '008000', '00cc00', 'b3ff1a', '99ff99', 'b30088', 'e68a00', 'ffad33', 'ffe0b3', 'ffff00', 'aa7700', 'd3bf9b', 'ffffff', '4780f3', '1b1716'], 'Land_Cover_class_values': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 'Land_Use_class_names': ['Agriculture', 'Developed', 'Forest', 'Non-Forest Wetland', 'Other', 'Rangeland or Pasture', 'No

In [15]:
# Abbreviated repeated steps from Module 4
# For a more detailed look at these steps, refer to module 4

# First, we will bring in LandTrendr data for each year

landTrendrCollection = ee.ImageCollection(export_landTrendr_collection)

landTrendrCollectionInfo = landTrendrCollection.first().toDictionary().getInfo()

# View exported LT output
# Convert stacked outputs into collection of fitted, magnitude, slope, duration, etc values for each year
# While the fitted LandTrendr value is generally of most importance to our models, 
# LandTrendr segment duration, slope, and magnitude of change can also help our models
lt_fit = changeDetectionLib.batchSimpleLTFit(landTrendrCollection,landTrendrCollectionInfo['startYear'],landTrendrCollectionInfo['endYear'],None,bandPropertyName='band',arrayMode=True)


# Bring in raw CCDC assets
ccdcTiles = ee.ImageCollection(export_ccdc_collection)
composites = ee.ImageCollection(export_composite_collection)

# CCDC date props
startYear = 1984
endYear = 2023
startJulian = 1
endJulian = 365

ccdcImg = ccdcTiles.mosaic()
#Specify which harmonics to use when predicting the CCDC model
#CCDC exports the first 3 harmonics (1 cycle/yr, 2 cycles/yr, and 3 cycles/yr)
#If you only want to see yearly patterns, specify [1]
#If you would like a tighter fit in the predicted value, include the second or third harmonic as well [1,2,3]
whichHarmonics = [1,2,3]

#Whether to fill gaps between segments' end year and the subsequent start year to the break date
fillGaps = False

# Proportion of a year segment can be extrapolated
# This can be important if the CCDC run had a break toward the end of the available raw data, but has not fit a 
# new model. This would introduce a null value past that break date if set to 0
# By setting to e.g. 0.3, a segment at the end can be extended by 0.3 of a year before it will result in a null value
tEndExtrapolationPeriod = 0.3


# If annualizeWithCompositeDates = False, this month and day will be used as the time to pull CCDC values
yearStartMonth = 4
yearStartDay = 1


# When annualizing CCDC outputs, we can pull the CCDC fitted, harmonic model coefficients, 
# and fitted difference from the previous year using a set date (e.g. April 1) or we can use the date from within our annual
# composites to increase the chance the CCDC output matches the timing of the composite values used within
# LandTrendr. In this example, we can bring in the annual composites and pull the dates for annualizing CCDC 
# by setting annualizeWithCompositeDates = True
# This will slow down computing, so you can set it to False for testing runs
annualizeWithCompositeDates = False
compositeCollection = composites

annualSegCoeffs = changeDetectionLib.annualizeCCDC(ccdcImg, startYear, endYear, startJulian, endJulian, tEndExtrapolationPeriod, 
  yearStartMonth, yearStartDay, annualizeWithCompositeDates, compositeCollection)
annualCCDC = changeDetectionLib.getFitSlopeCCDC(annualSegCoeffs, startYear, endYear)

# Bring in terrain data
terrainStack = []

# Can use any elevation data
# Will use SRTM for this example
# Resampling must be set to bicubic or bilinear or there will be artifacts in terrain derivatives
elevation = ee.Image("USGS/SRTMGL1_003").resample('bicubic')
aspect = ee.Terrain.aspect(elevation)
sinAspect = aspect.sin().rename('sinAspect')
cosAspect = aspect.cos().rename('cosAspect')

terrainStack.extend([sinAspect,cosAspect])

# A very simple algorithm for elevation position
# Good at finding ridges and depressions
def tpi(elevation,n):
    return elevation.subtract(elevation.focalMean(n)).rename([f'TPI_{int(n*2)}'])


# Get TPI for 11 and 21 pixel diameter circular kernels
for r in [5.5,10.5,20.5]:
    tpiR = tpi(elevation,r)
    terrainStack.append(tpiR)

# Stack all terrain data
terrainStack = ee.Image.cat(terrainStack)
print('Available terrain bands:',terrainStack.bandNames().getInfo())
print('Available LandTrendr Predictor Variables:',lt_fit.first().bandNames().getInfo())
print('Available CCDC Predictor Variables:',annualCCDC.first().bandNames().getInfo())

# Set up function to get predictor stack for a given year
def getPredictorStack(yr):
     # Filter the LandTrendr and CCDC data for the given year
    lt_yr = lt_fit.filter(ee.Filter.calendarRange(yr,yr,'year')).first()
    ccdc_yr = annualCCDC.filter(ee.Filter.calendarRange(yr,yr,'year')).first()

    # Get a stack of all predictor bands
    extractionStack = ee.Image.cat([lt_yr,ccdc_yr,terrainStack])
    return extractionStack.set('system:time_start',ee.Date.fromYMD(yr,6,1).millis())


# Get the field names for prediction
predictor_field_names = getPredictorStack(2015).bandNames().getInfo()

# Filter out any non null values (any training plot with missing predictor data will cause the model to fail entirely)
training_data = training_data.filter(ee.Filter.notNull(predictor_field_names))

print('Done')

Available terrain bands: ['sinAspect', 'cosAspect', 'TPI_11', 'TPI_21', 'TPI_41']
Available LandTrendr Predictor Variables: ['NBR_LT_dur', 'NBR_LT_fitted', 'NBR_LT_mag', 'NBR_LT_slope', 'NBR_LT_diff', 'NDVI_LT_dur', 'NDVI_LT_fitted', 'NDVI_LT_mag', 'NDVI_LT_slope', 'NDVI_LT_diff', 'brightness_LT_dur', 'brightness_LT_fitted', 'brightness_LT_mag', 'brightness_LT_slope', 'brightness_LT_diff', 'greenness_LT_dur', 'greenness_LT_fitted', 'greenness_LT_mag', 'greenness_LT_slope', 'greenness_LT_diff', 'nir_LT_dur', 'nir_LT_fitted', 'nir_LT_mag', 'nir_LT_slope', 'nir_LT_diff', 'red_LT_dur', 'red_LT_fitted', 'red_LT_mag', 'red_LT_slope', 'red_LT_diff', 'swir1_LT_dur', 'swir1_LT_fitted', 'swir1_LT_mag', 'swir1_LT_slope', 'swir1_LT_diff', 'swir2_LT_dur', 'swir2_LT_fitted', 'swir2_LT_mag', 'swir2_LT_slope', 'swir2_LT_diff', 'wetness_LT_dur', 'wetness_LT_fitted', 'wetness_LT_mag', 'wetness_LT_slope', 'wetness_LT_diff']
Available CCDC Predictor Variables: ['year', 'blue_CCDC_INTP', 'blue_CCDC_SLP', '

In [ ]:
# Repeated step from Module 4
# Now, we'll crosswalk the training fields to numeric codes
# The TimeSync fields are a string by default
# They must be a number for modeling
# Set up lookup dictionaries to convert the names to numeric codes
land_cover_name_code_dict = ee.Dictionary({'TREES':1,
                             'TSHRUBS-TRE':2,
                             'SHRUBS-TRE':3,
                             'GRASS-TREE':4,
                             'BARREN-TRE':5,
                             'TSHRUBS':6,
                             'SHRUBS':7,
                             'GRASS-SHRU':8,
                             'BARREN-SHR':9,
                             'GRASS':10,
                             'BARREN-GRA':11,
                             'BARREN-IMP':12,
                             'BARREN-IMP':12,
                             'WATER':14
                            })
land_use_name_code_dict = ee.Dictionary({'Agriculture':1,
                           'Developed':2,
                           'Forest':3,
                           'Non-forest Wetland':4,
                           'Other':5,
                           'Rangeland':6
                          })

change_code_dict = ee.Dictionary({'Debris': 3, 
                                  'Fire': 3, 
                                  'Growth/Recovery': 4, 
                                  'Harvest': 3, 'Hydrology': 3, 
                                  'Mechanical': 3, 
                                  'Other': 3, 
                                  'Spectral Decline': 2, 
                                  'Stable': 1, 
                                  'Structural Decline': 2, 
                                  'Wind/Ice': 3})

reference_field_dict = {'Land_Cover':{'field':'DOM_SEC_LC','name_code_dict':land_cover_name_code_dict},
                        'Land_Use':{'field':'DOM_LU','name_code_dict':land_use_name_code_dict},
                        'Change':{'field':'CP','name_code_dict':change_code_dict,
                                  'fields':['Slow Loss', 'Fast Loss', 'Gain']}
                       }
# Make a function that will get the code for a given name and set it
# We could also use the remap function to accomplish this
def set_class_code(plot,product):
    name_fieldName = reference_field_dict[product]['field']
    code_fieldName = ee.String(name_fieldName).cat('_Code')
    name = ee.String(plot.get(name_fieldName))
    code = reference_field_dict[product]['name_code_dict'].get(name)
    plot = plot.set(code_fieldName,code)
    return plot
                    
                    
    # print(name_fieldName,code_fieldName.getInfo(),name.getInfo(),code.getInfo())
            
# set_class_code(training_data.first(),'Land_Cover')
for product in list(reference_field_dict.keys()):
    print(product)
    training_data = training_data.map(lambda f:set_class_code(f,product))

print('Converted codes:',ee.Feature(training_data.first()).select(['CP','DOM_SEC_LC','DOM_LU','.*_Code']).toDictionary().getInfo())
print(training_data.aggregate_histogram('DOM_SEC_LC').getInfo())
print('Done')

In [ ]:
# Now, we'll explore the 


In [19]:
Map.clearMap()
# Not let's train and apply a RandomForest model

# Set some parameters
nTrees = 50 # Number of trees to use in the Random Forest model. > 50 can result in memory errors and generally doesn't help the model much
seed = 999 # Set a seed to ensure the same model is recreated with each run. Change this if a new one is needed
top_n_variables = 30 # Take this number of variables into the final model
apply_year = 2017 # Year to apply model to

# Get predictor stack for apply year
predictor_stack = getPredictorStack(apply_year)

rf_models = {}
def fit_rf(product_name):
    product_title = product_name.replace('_',' ')
    # Set up RF model 
    rf = ee.Classifier.smileRandomForest(numberOfTrees = nTrees, variablesPerSplit = None, minLeafPopulation = 1, bagFraction = 0.5, maxNodes = None, seed = seed)

    # Fit the RF model using all predictors
    rf = rf.train(training_data, reference_field_dict[product_name]['field']+'_Code', predictor_field_names)
    rf_models[product_name] = rf
    # Get model inf
    # importance = getModelInfo(rf, lcms_viz_dict[f'{product_name}_class_names'][:-1],f'All Predictors {product_title}')
    predicted = predictor_stack.classify(rf,product_name).set(lcms_viz_dict)

    Map.addLayer(predicted,{'autoViz':True},f'All Predictors {product_title} {apply_year}')

for product in ['Land_Cover','Land_Use','Change']:
    fit_rf(product)
# # importance = var_importance_plot(rf_lc_info['importance'],'Land Cover All Variables')
# rf_lc = ee.Classifier.smileRandomForest(numberOfTrees = nTrees, variablesPerSplit = None, minLeafPopulation = 1, bagFraction = 0.5, maxNodes = None, seed = seed)
# lc_predictor_field_names =  list(importance.keys())
# rf_lc = rf_lc.train(training_data, reference_field_dict['Land_Cover']['field']+'_Code',lc_predictor_field_names)#['NBR_LT_fitted'])
# # importance = rf_lc.explain().get('importance').getInfo()
# getModelInfo(rf_lc, lcms_viz_dict['Land_Cover_class_names'][:-1],f'Top {top_n_variables} Variables Land Cover')
# # var_importance_plot(importance,f'Land Cover Top {top_n_variables}')


# lc_predicted = predictor_stack.classify(rf_lc,'Land_Cover').set(lcms_viz_dict)
# # # print(lc_predicted.bandNames().getInfo())
# Map.addLayer(lc_predicted,{'autoViz':True},f'Land Cover Top {top_n_variables}')
Map.turnOnInspector()
# Map.view()

# Notice that there is quite a bit omission error in fast loss even for the training samples. 
# This means the omission error is likely much higher in reality

Adding layer: All Predictors Land Cover 2017
Adding layer: All Predictors Land Use 2017
Adding layer: All Predictors Change 2017


In [26]:
Map.clearMap()
# Now we will explore the different output modes available for GEE classifiers
# Random Forests supports many output modes
# From the GEE docs:
# CLASSIFICATION (default): The output is the class number.

# REGRESSION: The output is the result of standard regression.

# PROBABILITY: The output is the probability that the classification is correct.

# MULTIPROBABILITY: The output is an array of probabilities that each class is correct ordered by classes seen.

# RAW: The output is an array of the internal representation of the classification process. For example, the raw votes in multi-decision tree models.

# RAW_REGRESSION: The output is an array of the internal representation of the regression process. For example, the raw predictions of multiple regression trees.

# Not all classifiers support modes other than CLASSIFICATION.

# We will now apply the Land Cover model we fit earlier using CLASSIFICATION (which chooses the class with the most votes in RF)
# and then we will look at MULTIPROBABILITY (which stores the vote proportion for each class)

# Query the map to see how the two relate
# The MULTIPROBABILITY layer will yield a list

product_name = 'Land_Cover'
product_title = product_name.replace('_',' ')
rf = rf_models[product_name]
print('Current mode:',rf.mode().getInfo())
predicted = predictor_stack.classify(rf,product_name).set(lcms_viz_dict)

Map.addLayer(predicted,{'autoViz':True},f'CLASSIFICATION {product_title} {apply_year}')

rf = rf.setOutputMode('MULTIPROBABILITY')
print('Reset mode:',rf.mode().getInfo())
predicted = predictor_stack.classify(rf,product_name)

Map.addLayer(predicted,{},f'MULTIPROBABILITY {product_title} {apply_year}')

Map.turnOnInspector()
Map.view()

Current mode: CLASSIFICATION
Adding layer: CLASSIFICATION Land Cover 2017
Reset mode: MULTIPROBABILITY
Adding layer: MULTIPROBABILITY Land Cover 2017
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1235/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://23dcc4ff89e513fb-dot-us-west3.notebooks.googleusercontent.com/proxy/1235/geeView/?accessToken=None


127.0.0.1 - - [09/Aug/2023 18:30:15] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


In [31]:
Map.clearMap()
# Now lets export predicted assets
# First, we'll set up the study area and a tile to export across

studyArea = ee.FeatureCollection('projects/lcms-292214/assets/R8/PR_USVI/Ancillary/prusvi_boundary_buff2mile')

# Set the size (in meters) of the tiles
tileSize = 60000

# Set the projection
crs = getImagesLib.common_projections['NLCD_CONUS']['crs']
transform  = getImagesLib.common_projections['NLCD_CONUS']['transform']
scale = None
projection = ee.Projection(crs,transform)

# Set up years to apply models across
apply_years = list(range(1985,2022+1))

# Get the grid
grid = studyArea.geometry().coveringGrid(projection.atScale(tileSize))
Map.addLayer(grid,{},'Tile Grid {}m'.format(tileSize))

Map.centerObject(studyArea)
# Map.view()

Adding layer: Tile Grid 60000m


In [ ]:
# Adapted from Module 3
# Get individual tiles and apply models over apply years and tiles
ids = grid.aggregate_histogram('system:index').keys().getInfo()
for product_name in rf_models.keys():
    rf = rf_models[product_name]
    rf = rf.setOutputMode('MULTIPROBABILITY')
    for apply_year in apply_years:
        predictor_stack = getPredictorStack(apply_year)
        predicted = predictor_stack.classify(rf,product_name)
        predicted = predicted.set({'product':product_name,
                                   'year':apply_year,
                                   'study_area':'PRUSVI',
                                   'system:time_start':ee.Date.fromYMD(apply_year,6,1).millis()
                                  })
        for id in ids:
            id_title = id.replace(',','-')
            
            # Get the tile and buffer it so there are no missing pixels at tile edges
            tile = grid.filter(ee.Filter.eq('system:index',id)).geometry().intersection(studyArea,240,projection).buffer(900)
            predicted = predicted.set({'TileSize':tileSize,
                     'TileID':id_title})
    
    
            exportName = f'LCMS_Raw_{product_name}_Tile-{tileSize}m_ID{id_title}_yr{apply_year}'
            exportPath = f'{export_rawLCMSOutputs_collection}/{exportName}'
            print(exportPath)

            getImagesLib.exportToAssetWrapper(predicted,exportName,exportPath,{'.default':'sample'},tile,scale,crs,transform,overwrite=False)


projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs/LCMS_Raw_Land_Cover_Tile-60000m_ID90-54_yr1985
pyramiding object: {'.default': 'sample'}
LCMS_Raw_Land_Cover_Tile-60000m_ID90-54_yr1985 currently exists or is being exported and overwrite = False. Export not started. Set overwite = True if you would like to overwite any existing asset or asset exporting task
projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs/LCMS_Raw_Land_Cover_Tile-60000m_ID91-52_yr1985
pyramiding object: {'.default': 'sample'}
LCMS_Raw_Land_Cover_Tile-60000m_ID91-52_yr1985 currently exists or is being exported and overwrite = False. Export not started. Set overwite = True if you would like to overwite any existing asset or asset exporting task
projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs/LCMS_Raw_Land_Cover_Tile-60000m_ID91-53_yr1985
pyramiding object: {'.default': 'sample'}
Exporting: LCMS_Raw_Land_Cover_Tile-60000m_ID91-53_yr1985
<Task

127.0.0.1 - - [09/Aug/2023 19:04:55] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


Exporting: LCMS_Raw_Land_Cover_Tile-60000m_ID93-52_yr2021
<Task EXPORT_IMAGE: LCMS_Raw_Land_Cover_Tile-60000m_ID93-52_yr2021 (UNSUBMITTED)>
projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs/LCMS_Raw_Land_Cover_Tile-60000m_ID93-53_yr2021
pyramiding object: {'.default': 'sample'}
Exporting: LCMS_Raw_Land_Cover_Tile-60000m_ID93-53_yr2021
<Task EXPORT_IMAGE: LCMS_Raw_Land_Cover_Tile-60000m_ID93-53_yr2021 (UNSUBMITTED)>
projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs/LCMS_Raw_Land_Cover_Tile-60000m_ID94-52_yr2021
pyramiding object: {'.default': 'sample'}
Exporting: LCMS_Raw_Land_Cover_Tile-60000m_ID94-52_yr2021
<Task EXPORT_IMAGE: LCMS_Raw_Land_Cover_Tile-60000m_ID94-52_yr2021 (UNSUBMITTED)>
projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs/LCMS_Raw_Land_Cover_Tile-60000m_ID95-51_yr2021
pyramiding object: {'.default': 'sample'}
Exporting: LCMS_Raw_Land_Cover_Tile-60000m_ID95-51_yr2021
<Task EXPORT_IMAGE: LCMS_Ra

127.0.0.1 - - [09/Aug/2023 19:22:36] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


Exporting: LCMS_Raw_Land_Use_Tile-60000m_ID96-53_yr2017
<Task EXPORT_IMAGE: LCMS_Raw_Land_Use_Tile-60000m_ID96-53_yr2017 (UNSUBMITTED)>
projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs/LCMS_Raw_Land_Use_Tile-60000m_ID90-54_yr2018
pyramiding object: {'.default': 'sample'}
Exporting: LCMS_Raw_Land_Use_Tile-60000m_ID90-54_yr2018
<Task EXPORT_IMAGE: LCMS_Raw_Land_Use_Tile-60000m_ID90-54_yr2018 (UNSUBMITTED)>
projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs/LCMS_Raw_Land_Use_Tile-60000m_ID91-52_yr2018
pyramiding object: {'.default': 'sample'}
Exporting: LCMS_Raw_Land_Use_Tile-60000m_ID91-52_yr2018
<Task EXPORT_IMAGE: LCMS_Raw_Land_Use_Tile-60000m_ID91-52_yr2018 (UNSUBMITTED)>
projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs/LCMS_Raw_Land_Use_Tile-60000m_ID91-53_yr2018
pyramiding object: {'.default': 'sample'}
Exporting: LCMS_Raw_Land_Use_Tile-60000m_ID91-53_yr2018
<Task EXPORT_IMAGE: LCMS_Raw_Land_Use_Tile-6000

...

Exporting: LCMS_Raw_Change_Tile-60000m_ID92-53_yr2003
<Task EXPORT_IMAGE: LCMS_Raw_Change_Tile-60000m_ID92-53_yr2003 (UNSUBMITTED)>
projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs/LCMS_Raw_Change_Tile-60000m_ID93-52_yr2003
pyramiding object: {'.default': 'sample'}
Exporting: LCMS_Raw_Change_Tile-60000m_ID93-52_yr2003
<Task EXPORT_IMAGE: LCMS_Raw_Change_Tile-60000m_ID93-52_yr2003 (UNSUBMITTED)>
projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs/LCMS_Raw_Change_Tile-60000m_ID93-53_yr2003
pyramiding object: {'.default': 'sample'}
Exporting: LCMS_Raw_Change_Tile-60000m_ID93-53_yr2003
<Task EXPORT_IMAGE: LCMS_Raw_Change_Tile-60000m_ID93-53_yr2003 (UNSUBMITTED)>
projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs/LCMS_Raw_Change_Tile-60000m_ID94-52_yr2003
pyramiding object: {'.default': 'sample'}
Exporting: LCMS_Raw_Change_Tile-60000m_ID94-52_yr2003
<Task EXPORT_IMAGE: LCMS_Raw_Change_Tile-60000m_ID94-52_yr2003 (UNS

127.0.0.1 - - [09/Aug/2023 19:43:00] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


Exporting: LCMS_Raw_Change_Tile-60000m_ID95-53_yr2006
<Task EXPORT_IMAGE: LCMS_Raw_Change_Tile-60000m_ID95-53_yr2006 (UNSUBMITTED)>
projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs/LCMS_Raw_Change_Tile-60000m_ID96-52_yr2006
pyramiding object: {'.default': 'sample'}
Exporting: LCMS_Raw_Change_Tile-60000m_ID96-52_yr2006
<Task EXPORT_IMAGE: LCMS_Raw_Change_Tile-60000m_ID96-52_yr2006 (UNSUBMITTED)>
projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs/LCMS_Raw_Change_Tile-60000m_ID96-53_yr2006
pyramiding object: {'.default': 'sample'}
Exporting: LCMS_Raw_Change_Tile-60000m_ID96-53_yr2006
<Task EXPORT_IMAGE: LCMS_Raw_Change_Tile-60000m_ID96-53_yr2006 (UNSUBMITTED)>
projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs/LCMS_Raw_Change_Tile-60000m_ID90-54_yr2007
pyramiding object: {'.default': 'sample'}
Exporting: LCMS_Raw_Change_Tile-60000m_ID90-54_yr2007
<Task EXPORT_IMAGE: LCMS_Raw_Change_Tile-60000m_ID90-54_yr2007 (UNS

127.0.0.1 - - [09/Aug/2023 19:45:02] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


Exporting: LCMS_Raw_Change_Tile-60000m_ID96-53_yr2008
<Task EXPORT_IMAGE: LCMS_Raw_Change_Tile-60000m_ID96-53_yr2008 (UNSUBMITTED)>
projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs/LCMS_Raw_Change_Tile-60000m_ID90-54_yr2009
pyramiding object: {'.default': 'sample'}
Exporting: LCMS_Raw_Change_Tile-60000m_ID90-54_yr2009
<Task EXPORT_IMAGE: LCMS_Raw_Change_Tile-60000m_ID90-54_yr2009 (UNSUBMITTED)>
projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs/LCMS_Raw_Change_Tile-60000m_ID91-52_yr2009
pyramiding object: {'.default': 'sample'}
Exporting: LCMS_Raw_Change_Tile-60000m_ID91-52_yr2009
<Task EXPORT_IMAGE: LCMS_Raw_Change_Tile-60000m_ID91-52_yr2009 (UNSUBMITTED)>
projects/rcr-gee/assets/lcms-training/lcms-training_module-5_rawLCMSOutputs/LCMS_Raw_Change_Tile-60000m_ID91-53_yr2009
pyramiding object: {'.default': 'sample'}
Exporting: LCMS_Raw_Change_Tile-60000m_ID91-53_yr2009
<Task EXPORT_IMAGE: LCMS_Raw_Change_Tile-60000m_ID91-53_yr2009 (UNS

In [ ]:
# Can track tasks here or at https://code.earthengine.google.com/tasks
# If you'd like to track the tasks, use this:
# tml.trackTasks2()

# If you want to cancel all running tasks, you can use this function
# tml.batchCancel()

# If you want to empty the collection of all images
# aml.batchDelete(export_rawLCMSOutputs_collection, type = 'imageCollection')

print('done')